<a href="https://colab.research.google.com/github/gevenbly/TensorAlgs/blob/main/network_render.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib import patches
from typing import Optional, List, Union, Tuple
from network_contract import pre_ncon

In [10]:
def draw_network(curr_fig, connects, names=None, coords=None, cols=None, 
                 dims=None, circ_rad=0.3, fontsize=16, tagsize=8, subplot=111, 
                 draw_labels=True, title=None, order=None, bkg_col='silver',
                 col_inds=True, legend_extend=1.5):

  # recast connects in canonical form, compute contraction list and costs
  (nm_connects, fwd_dict, back_dict, pt_cont, bn_cont, pt_costs, 
   bn_costs) = pre_ncon(connects, dims, order)

  # configuration parameters
  double_offset = 0.09 # spacing between double indices
  triple_offset = 0.125 # spacing between triple indices
  tol = 1e-8 # tolerance for float comparison of coords

  # initialize figure
  ax1 = curr_fig.add_subplot(subplot, aspect='equal')

  # make adjacency matrix
  adjmat = _ncon_to_adjmat(connects)
  N = adjmat.shape[0]
  
  # make palettes for tensors and for markers
  tensor_palette = ['mediumspringgreen', 'cornflowerblue', 'plum', 'lightskyblue',
              'turquoise', 'moccasin', 'yellowgreen', 'lightgreen', 'sandybrown',
              'lightsalmon', 'lightcoral', 'silver']
  num_tcols = len(tensor_palette)
  marker_palette = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 
                    'violet']
  num_mcols = len(marker_palette)
  marker_set = ['o', '^', 'x', 's']
  line_set = ['solid', 'dashed', 'dashdot', 'dotted']
  
  # generate default names
  if names is None:
    names = [f"T{lab}" for lab in range(N)]

  # generate default tensor colors
  if cols is None:
    unique_names = []
    for name in names:
      if name not in unique_names:
          unique_names.append(name)
    name_dict = dict(zip(unique_names, np.arange(len(unique_names))))
    cols = [name_dict[name] for name in names]
  
  # generate default coords
  if coords is None:
    R = 1.5 * N / (2 * np.pi)
    angle_space = 2 * np.pi / N
    coords = [0] * N
    init_angle = 0
    for k in range(N):
      thet = k * angle_space + init_angle
      coords[k] = (-R * np.cos(thet), R * np.sin(thet))

  # draw closed indices  
  x_residue = np.zeros(N, dtype=np.float)
  y_residue = np.zeros(N, dtype=np.float)
  for k in range(N):
    for p in range(k):
      if (adjmat[k,p] > 0):
        # logic for determining sign of angle
        if (coords[p][0] == coords[k][0]):
          temp_order = coords[p][1] < coords[k][1]
        elif (coords[p][0] < coords[k][0]):
          temp_order = coords[p][1] <= coords[k][1]
        else:
          temp_order = coords[p][1] < coords[k][1]

        # find angle between tensors
        if np.abs(coords[p][1] - coords[k][1]) < tol:
          thet = np.pi/2
        else:
          thet = np.arctan((coords[p][0]-coords[k][0]) / 
                          (coords[p][1]-coords[k][1]))
          
        # find coordinates between centers of connected tensors 
        x0 = coords[k][0] + ((-1)**temp_order) * np.sin(thet) * circ_rad
        x1 = coords[p][0] - ((-1)**temp_order) * np.sin(thet) * circ_rad
        y0 = coords[k][1] + ((-1)**temp_order) * np.cos(thet) * circ_rad
        y1 = coords[p][1] - ((-1)**temp_order) * np.cos(thet) * circ_rad
        
        # store residues for determining angles of open indices 
        x_residue[k] = x_residue[k] + ((-1)**temp_order) * np.sin(thet) * circ_rad
        x_residue[p] = x_residue[p] - ((-1)**temp_order) * np.sin(thet) * circ_rad
        y_residue[k] = y_residue[k] + ((-1)**temp_order) * np.cos(thet) * circ_rad
        y_residue[p] = y_residue[p] - ((-1)**temp_order) * np.cos(thet) * circ_rad

        # determine index colors and linetypes
        if col_inds:
          cont_inds = np.intersect1d(nm_connects[k], nm_connects[p])
          for count, inds in enumerate(bn_cont):
            if len(np.intersect1d(inds, cont_inds)) > 0:
              ind_line, ind_col = np.divmod(count, num_mcols)
              line_col = marker_palette[ind_col]
              line_type = line_set[ind_line]
              break
        else:
          line_col = 'k'
          line_type = 'solid'

        if adjmat[k,p] == 1:
          # determine tag colors and shapes
          ind_names, pos0, pos1 = np.intersect1d(connects[k],connects[p], 
                                                 return_indices=True)
          mark0, col0 = np.divmod(pos0.item(), num_mcols)
          mark1, col1 = np.divmod(pos1.item(), num_mcols)

          # draw the index
          _draw_index(x0, x1, y0, y1,
                      color=line_col, line_type=line_type,
                      color0=marker_palette[col0], 
                      color1=marker_palette[col1],  
                      marker0=marker_set[mark0], marker1=marker_set[mark1],
                      markersize=tagsize, t_name=ind_names.item(), 
                      fontsize=fontsize, draw_labels=draw_labels, ax1=ax1,
                      bkg_col=bkg_col)
          
        elif adjmat[k,p] == 2:
          # determine tag colors and shapes
          ind_names, pos0, pos1 = np.intersect1d(connects[k],connects[p], 
                                                 return_indices=True)
          mark00, col00 = np.divmod(pos0[0].item(), num_mcols)
          mark01, col01 = np.divmod(pos0[1].item(), num_mcols)
          mark10, col10 = np.divmod(pos1[0].item(), num_mcols)
          mark11, col11 = np.divmod(pos1[1].item(), num_mcols)

          # determine index end points (perp shifted due to doubling)
          x0p = x0 + np.cos(thet)*double_offset
          x0m = x0 - np.cos(thet)*double_offset
          y0p = y0 - np.sin(thet)*double_offset
          y0m = y0 + np.sin(thet)*double_offset

          x1p = x1 + np.cos(thet)*double_offset
          x1m = x1 - np.cos(thet)*double_offset
          y1p = y1 - np.sin(thet)*double_offset
          y1m = y1 + np.sin(thet)*double_offset

          # draw the indices
          _draw_index(x0m, x1m, y0m, y1m,
                      color=line_col, line_type=line_type,
                      color0=marker_palette[col00], 
                      color1=marker_palette[col10], 
                      marker0=marker_set[mark00], marker1=marker_set[mark10],
                      markersize=tagsize, t_name=ind_names[0].item(), 
                      fontsize=fontsize, draw_labels=draw_labels, ax1=ax1,
                      bkg_col=bkg_col)

          _draw_index(x0p, x1p, y0p, y1p,
                      color=line_col, line_type=line_type,
                      color0=marker_palette[col01], 
                      color1=marker_palette[col11], 
                      marker0=marker_set[mark01], marker1=marker_set[mark11],
                      markersize=tagsize, t_name=ind_names[1].item(), 
                      fontsize=fontsize, draw_labels=draw_labels, ax1=ax1,
                      bkg_col=bkg_col)
        
        elif adjmat[k,p] == 3:
          # determine tag colors and shapes
          ind_names, pos0, pos1 = np.intersect1d(connects[k],connects[p], 
                                                 return_indices=True)
          mark00, col00 = np.divmod(pos0[0].item(), num_mcols)
          mark01, col01 = np.divmod(pos0[1].item(), num_mcols)
          mark02, col02 = np.divmod(pos0[2].item(), num_mcols)
          mark10, col10 = np.divmod(pos1[0].item(), num_mcols)
          mark11, col11 = np.divmod(pos1[1].item(), num_mcols)
          mark12, col12 = np.divmod(pos1[2].item(), num_mcols)

          # determine index end points (perp shifted due to doubling)
          x0p = x0 + np.cos(thet)*triple_offset
          x0m = x0 - np.cos(thet)*triple_offset
          y0p = y0 - np.sin(thet)*triple_offset
          y0m = y0 + np.sin(thet)*triple_offset

          x1p = x1 + np.cos(thet)*triple_offset
          x1m = x1 - np.cos(thet)*triple_offset
          y1p = y1 - np.sin(thet)*triple_offset
          y1m = y1 + np.sin(thet)*triple_offset

          # draw the indices
          _draw_index(x0m, x1m, y0m, y1m, 
                      color=line_col, line_type=line_type,
                      color0=marker_palette[col00], 
                      color1=marker_palette[col10], 
                      marker0=marker_set[mark00], marker1=marker_set[mark10],
                      markersize=tagsize, t_name=ind_names[0].item(), 
                      fontsize=0.8*fontsize, draw_labels=draw_labels, ax1=ax1,
                      bkg_col=bkg_col)
          
          _draw_index(x0, x1, y0, y1,
                      color=line_col, line_type=line_type,
                      color0=marker_palette[col01], 
                      color1=marker_palette[col11], 
                      marker0=marker_set[mark01], marker1=marker_set[mark11],
                      markersize=tagsize, t_name=ind_names[1].item(), 
                      fontsize=0.8*fontsize, draw_labels=draw_labels, ax1=ax1,
                      bkg_col=bkg_col)

          _draw_index(x0p, x1p, y0p, y1p,
                      color=line_col, line_type=line_type,
                      color0=marker_palette[col02], 
                      color1=marker_palette[col12], 
                      marker0=marker_set[mark02], marker1=marker_set[mark12],
                      markersize=tagsize, t_name=ind_names[2].item(), 
                      fontsize=0.8*fontsize, draw_labels=draw_labels, ax1=ax1,
                      bkg_col=bkg_col)
        
        else:
          _draw_index(x0, x1, y0, y1,
                      color=line_col, line_type=line_type,
                      color0='k', 
                      color1='k',  
                      marker0='o', marker1='o',
                      markersize=tagsize, 
                      fontsize=fontsize, draw_labels=False, ax1=ax1,
                      bkg_col=bkg_col, linewidth=5)

  # draw open indices  
  for k in range(N):
    if adjmat[k,k] > 0:
      # normalize the residue from the closed indices
      res_vec = np.array([x_residue[k], y_residue[k]])
      res_norm = np.linalg.norm(res_vec)
      if res_norm > tol:
        res_vec = res_vec / res_norm
      else:
        # set free indices to point downwards if no residue
        res_vec = np.array([0, -1])

      # logic for determining residue signs
      coordx = coords[k][0] - res_vec[0]
      coordy = coords[k][1] - res_vec[1]
      if (coordx == coords[k][0]):
        temp_order = coordy < coords[k][1]
      elif (coordx < coords[k][0]):
        temp_order = coordy <= coords[k][1]
      else:
        temp_order = coordy < coords[k][1]

      # angle of open index orientation
      if coordy == coords[k][1]:
        thet = np.pi/2
      else:
        thet = np.arctan((coordx-coords[k][0]) / (coordy-coords[k][1]))
      
      # determine end points of markers
      x0 = coords[k][0] + ((-1)**temp_order) * np.sin(thet) * circ_rad
      x1 = coordx - ((-1)**temp_order) * np.sin(thet) * circ_rad
      y0 = coords[k][1] + ((-1)**temp_order) * np.cos(thet) * circ_rad
      y1 = coordy - ((-1)**temp_order) * np.cos(thet) * circ_rad

      if adjmat[k,k] == 1:
        # determine tag colors and shapes
        loc = np.where(np.array(nm_connects[k], dtype=int) <= 0)[0].item()
        val = np.abs(nm_connects[k][loc]) - 1
        mark0, col0 = np.divmod(loc, num_mcols)
        mark1, col1 = np.divmod(val, num_mcols)
        xf = (coordx + x1) / 2
        yf = (coordy + y1) / 2

        # draw the index
        _draw_index(x0, x1, y0, y1, xf=xf, yf=yf,
                    color0=marker_palette[col0], 
                    color1=marker_palette[col1],  
                    marker0=marker_set[mark0], marker1=marker_set[mark1],
                    markersize=tagsize, t_name=connects[k][loc], 
                    fontsize=fontsize, draw_labels=draw_labels, ax1=ax1,
                    bkg_col=bkg_col)
      
      elif adjmat[k,k] == 2:
        # determine tag end points
        x0p = x0 + np.cos(thet)*double_offset
        x0m = x0 - np.cos(thet)*double_offset
        y0p = y0 - np.sin(thet)*double_offset
        y0m = y0 + np.sin(thet)*double_offset

        x1p = x1 + np.cos(thet)*double_offset
        x1m = x1 - np.cos(thet)*double_offset
        y1p = y1 - np.sin(thet)*double_offset
        y1m = y1 + np.sin(thet)*double_offset

        x2 = (coordx + x1) / 2 
        y2 = (coordy + y1) / 2

        # determine marker shapes and colors
        loc = np.where(np.array(nm_connects[k], dtype=int) <= 0)[0]
        val0 = np.abs(nm_connects[k][loc[0]]) - 1
        val1 = np.abs(nm_connects[k][loc[1]]) - 1
        mark0A, col0A = np.divmod(loc[0].item(), num_mcols)
        mark0B, col0B = np.divmod(loc[1].item(), num_mcols)
        mark1A, col1A = np.divmod(val0, num_mcols)
        mark1B, col1B = np.divmod(val1, num_mcols)

        # draw the indices
        _draw_index(x0m, x1m, y0m, y1m,
                    xf=x2 - np.cos(thet)*double_offset,
                    yf=y2 + np.sin(thet)*double_offset,
                    color0=marker_palette[col0A], 
                    color1=marker_palette[col1A],  
                    marker0=marker_set[mark0A], marker1=marker_set[mark1A],
                    markersize=tagsize, t_name=connects[k][loc[0]],
                    fontsize=fontsize, draw_labels=draw_labels, ax1=ax1,
                    bkg_col=bkg_col)
        
        _draw_index(x0p, x1p, y0p, y1p,
                    xf=x2 + np.cos(thet)*double_offset,
                    yf=y2 - np.sin(thet)*double_offset,
                    color0=marker_palette[col0B], 
                    color1=marker_palette[col1B],  
                    marker0=marker_set[mark0B], marker1=marker_set[mark1B],
                    markersize=tagsize, t_name=connects[k][loc[1]],
                    fontsize=fontsize, draw_labels=draw_labels, ax1=ax1,
                    bkg_col=bkg_col)
        
      elif adjmat[k,k] == 3:
        # determine tag end points
        x0p = x0 + np.cos(thet)*triple_offset
        x0m = x0 - np.cos(thet)*triple_offset
        y0p = y0 - np.sin(thet)*triple_offset
        y0m = y0 + np.sin(thet)*triple_offset

        x1p = x1 + np.cos(thet)*triple_offset
        x1m = x1 - np.cos(thet)*triple_offset
        y1p = y1 - np.sin(thet)*triple_offset
        y1m = y1 + np.sin(thet)*triple_offset
        
        x2 = (coordx + x1) / 2 
        y2 = (coordy + y1) / 2

        # determine marker shapes and colors
        loc = np.where(np.array(nm_connects[k], dtype=int) <= 0)[0]
        val0 = np.abs(nm_connects[k][loc[0]]) - 1
        val1 = np.abs(nm_connects[k][loc[1]]) - 1
        val2 = np.abs(nm_connects[k][loc[2]]) - 1
        mark0A, col0A = np.divmod(loc[0].item(), num_mcols)
        mark0B, col0B = np.divmod(loc[1].item(), num_mcols)
        mark0C, col0C = np.divmod(loc[2].item(), num_mcols)
        mark1A, col1A = np.divmod(val0, num_mcols)
        mark1B, col1B = np.divmod(val1, num_mcols)
        mark1C, col1C = np.divmod(val2, num_mcols)

        # draw the indices
        _draw_index(x0m, x1m, y0m, y1m,
                    xf=x2 - np.cos(thet)*double_offset,
                    yf=y2 + np.sin(thet)*double_offset,
                    color0=marker_palette[col0A], 
                    color1=marker_palette[col1A],  
                    marker0=marker_set[mark0A], marker1=marker_set[mark1A],
                    markersize=tagsize, t_name=connects[k][loc[0]],
                    fontsize=0.8*fontsize, draw_labels=draw_labels, ax1=ax1,
                    bkg_col=bkg_col)
        
        _draw_index(x0, x1, y0, y1, xf=x2, yf=y2, 
                    color0=marker_palette[col0B], 
                    color1=marker_palette[col1B],  
                    marker0=marker_set[mark0B], marker1=marker_set[mark1B],
                    markersize=tagsize, t_name=connects[k][loc[1]],
                    fontsize=0.8*fontsize, draw_labels=draw_labels, ax1=ax1,
                    bkg_col=bkg_col)
        
        _draw_index(x0p, x1p, y0p, y1p,
                    xf=x2 + np.cos(thet)*double_offset,
                    yf=y2 - np.sin(thet)*double_offset,
                    color0=marker_palette[col0C], 
                    color1=marker_palette[col1C],  
                    marker0=marker_set[mark0C], marker1=marker_set[mark1C],
                    markersize=tagsize, t_name=connects[k][loc[2]],
                    fontsize=0.8*fontsize, draw_labels=draw_labels, ax1=ax1,
                    bkg_col=bkg_col)
            
      else:
        _draw_index(x0, x1, y0, y1,
                    color='k', line_type='solid',
                    color0='k', 
                    color1='k',  
                    marker0='o', marker1='o',
                    markersize=tagsize, 
                    fontsize=fontsize, draw_labels=False, ax1=ax1,
                    bkg_col=bkg_col, linewidth=5)

  # draw tensors
  for k in range(N):
    col_lab = np.mod(cols[k], num_tcols)
    _draw_tensor(ax1, center=coords[k], radius=circ_rad, 
                 color=tensor_palette[col_lab], 
                 name=names[k], fontsize=1.2*fontsize)

  # make title
  if title is not None:
    fontdict = {'fontsize': fontsize,
                'fontweight' : 'bold',
                'verticalalignment': 'baseline',
                'horizontalalignment': 'center'}
    plt.title(title, fontdict=fontdict, loc='center', pad=30)

  # extend the figure to make room for the legend (hacky...)
  if col_inds:
    xmax = max([coord[0] for coord in coords])
    ymax = max([coord[1] for coord in coords])
    plt.plot([xmax, xmax + legend_extend], [ymax, ymax], color=bkg_col,
             linewidth=0, linestyle='solid')

  # make plot
  curr_fig.patch.set_facecolor(bkg_col)
  plt.axis('off')
  plt.axis('scaled')

  # make legend
  if col_inds:
    all_lines = []
    for count, cost in enumerate(bn_costs):
      ind_line, ind_col = np.divmod(count, num_mcols)
      line_col = marker_palette[ind_col]
      line_type = line_set[ind_line]
      if not isinstance(cost, str):
        cost = "{:.2e}".format(cost)
      all_lines.append(mlines.Line2D([], [], color=line_col, linewidth=3,
                                     linestyle=line_type, label=cost))
    
    legend = plt.legend(handles=all_lines, title="costs:",
                        fancybox=True, fontsize=0.8*fontsize, 
                        title_fontsize=fontsize, loc='center right')
    frame = legend.get_frame()
    frame.set_color('darkgrey')
    legend.get_frame().set_edgecolor('k')

In [ ]:
def _draw_tensor(ax1, center=(0,0), radius=0.5, color='white', fontsize=12, 
                name=None):
  
  ax1.add_patch(patches.Circle(center,
                               radius=radius,
                               color=color, 
                               linewidth=0, 
                               fill=True))
  
  ax1.add_patch(patches.Circle(center,
                               radius=radius,
                               color='k', 
                               linewidth=2, 
                               fill=False))
  if name is not None:
    t = ax1.text(center[0], center[1], name, fontsize=fontsize, 
             horizontalalignment='center',
             verticalalignment='center',
             color='white')
    t.set_path_effects([path_effects.Stroke(linewidth=2, foreground='black'),
                       path_effects.Normal()])

In [ ]:
def _draw_index(x0, x1, y0, y1, xf=None, yf=None, color='k', color0='k', 
                color1='k', line_type='solid', marker0='o', marker1='o', 
                markersize=8, t_name='', fontsize=16, draw_labels=True, 
                ax1=None, bkg_col='w', linewidth=2):
  
  # generate coords for index label
  if xf is None:
    xf = (x0 + x1) / 2
  if yf is None:
    yf = (y0 + y1) / 2
  
  # plot indices
  plt.plot([x0, x1], [y0, y1], color=color, linewidth=linewidth,
           linestyle=line_type)

  # plot 1st endpoint tags
  plt.plot([x0], [y0], marker=marker0, markersize=markersize + 2, color='k')
  plt.plot([x0], [y0], marker=marker0, markersize=markersize, color=color0)
  
  # plot 2nd endpoint tags
  plt.plot([x1], [y1], marker=marker1, markersize=markersize + 2, color='k')
  plt.plot([x1], [y1], marker=marker1, markersize=markersize, color=color1)
  
  # plot index label
  if draw_labels:
    t = ax1.text(xf, yf, t_name, 
                  fontsize=fontsize, 
                  horizontalalignment='center',
                  verticalalignment='center',
                  color='black')
    t.set_path_effects([path_effects.Stroke(linewidth=4, foreground=bkg_col),
                      path_effects.Normal()])

In [ ]:
def _ncon_to_adjmat(labels: List[List[int]]):
  # process inputs
  N = len(labels)
  ranks = [len(labels[i]) for i in range(N)]
  flat_labels = np.hstack([labels[i] for i in range(N)])
  tensor_counter = np.hstack(
      [i * np.ones(ranks[i], dtype=int) for i in range(N)])
  index_counter = np.hstack([np.arange(ranks[i]) for i in range(N)])

  # build log-adjacency index-by-index
  adjmat = np.zeros([N, N], dtype=int)
  unique_labels = np.unique(flat_labels)
  for ele in unique_labels:
    # identify tensor/index location of each edge
    tnr = tensor_counter[flat_labels == ele]
    ind = index_counter[flat_labels == ele]
    if len(ind) == 1:  # external index
      adjmat[tnr[0], tnr[0]] += 1
    elif len(ind) == 2:  # internal index
      if tnr[0] != tnr[1]:  # ignore partial traces
        adjmat[tnr[0], tnr[1]] += 1
        adjmat[tnr[1], tnr[0]] += 1

  return adjmat